In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [3]:
df = pd.read_csv("../data/03_feature_engineered/2021-2022_engineered.csv")
df['game'] = df['home_team'] + df['away_team']
df.set_index(['date','time', 'game'], inplace=True)

In [4]:
df

round        home_team  \
date       time  game                                                          
2021-08-13 20:00 BrentfordArsenal               Matchweek 1        Brentford   
2021-08-14 12:30 Manchester UtdLeeds United     Matchweek 1   Manchester Utd   
           15:00 BurnleyBrighton                Matchweek 1          Burnley   
                 ChelseaCrystal Palace          Matchweek 1          Chelsea   
                 EvertonSouthampton             Matchweek 1          Everton   
...                                                     ...              ...   
2022-05-22 16:00 Crystal PalaceManchester Utd  Matchweek 38   Crystal Palace   
                 Leicester CitySouthampton     Matchweek 38   Leicester City   
                 LiverpoolWolves               Matchweek 38        Liverpool   
                 Manchester CityAston Villa    Matchweek 38  Manchester City   
                 Norwich CityTottenham         Matchweek 38     Norwich City   

                                                    away_team venue  result  \
date       time  game                                                         
2021-08-13 20:00 BrentfordArsenal                     Arsenal  Home       1   
2021-08-14 12:30 Manchester UtdLeeds United      Leeds United  Home       1   
           15:00 BurnleyBrighton                     Brighton  Home      -1   
                 ChelseaCrystal Palace         Crystal Palace  Home       1   
                 EvertonSouthampton               Southampton  Home       1   
...                                                       ...   ...     ...   
2022-05-22 16:00 Crystal PalaceManchester Utd  Manchester Utd  Home       1   
                 Leicester CitySouthampton        Southampton  Home       1   
                 LiverpoolWolves                       Wolves  Home       1   
                 Manchester CityAston Villa       Aston Villa  Home       1   
                 Norwich CityTottenham              Tottenham  Home      -1   

                                               home_goals  away_goals  \
date       time  game                                                   
2021-08-13 20:00 BrentfordArsenal                       2           0   
2021-08-14 12:30 Manchester UtdLeeds United             5           1   
           15:00 BurnleyBrighton                        1           2   
                 ChelseaCrystal Palace                  3           0   
                 EvertonSouthampton                     3           1   
...                                                   ...         ...   
2022-05-22 16:00 Crystal PalaceManchester Utd           1           0   
                 Leicester CitySouthampton              4           1   
                 LiverpoolWolves                        3           1   
                 Manchester CityAston Villa             3           2   
                 Norwich CityTottenham                  0           5   

                                               home_poss  away_poss  home_xg  \
date       time  game                                                          
2021-08-13 20:00 BrentfordArsenal                   36.0       64.0      1.2   
2021-08-14 12:30 Manchester UtdLeeds United         49.0       51.0      1.5   
           15:00 BurnleyBrighton                    37.0       63.0      1.5   
                 ChelseaCrystal Palace              62.0       38.0      0.7   
                 EvertonSouthampton                 48.0       52.0      2.4   
...                                                  ...        ...      ...   
2022-05-22 16:00 Crystal PalaceManchester Utd       39.0       61.0      0.6   
                 Leicester CitySouthampton          51.0       49.0      3.4   
                 LiverpoolWolves                    63.0       37.0      2.7   
                 Manchester CityAston Villa         71.0       29.0      3.4   
                 Norwich CityTottenham              41.0       59.0      0.

In [5]:
df_2021_2022 = df.copy()

In [6]:
# drop columns about in-game stats 
df_2021_2022.drop(columns=['home_goals', 'away_goals', 'home_poss', 'away_poss', 'home_xg', 
                           'away_xg', 'home_sh', 'away_sh', 'home_shot_on_target', 'away_shot_on_target'], inplace=True)

In [7]:
df_2021_2022

round        home_team  \
date       time  game                                                          
2021-08-13 20:00 BrentfordArsenal               Matchweek 1        Brentford   
2021-08-14 12:30 Manchester UtdLeeds United     Matchweek 1   Manchester Utd   
           15:00 BurnleyBrighton                Matchweek 1          Burnley   
                 ChelseaCrystal Palace          Matchweek 1          Chelsea   
                 EvertonSouthampton             Matchweek 1          Everton   
...                                                     ...              ...   
2022-05-22 16:00 Crystal PalaceManchester Utd  Matchweek 38   Crystal Palace   
                 Leicester CitySouthampton     Matchweek 38   Leicester City   
                 LiverpoolWolves               Matchweek 38        Liverpool   
                 Manchester CityAston Villa    Matchweek 38  Manchester City   
                 Norwich CityTottenham         Matchweek 38     Norwich City   

                                                    away_team venue  result  \
date       time  game                                                         
2021-08-13 20:00 BrentfordArsenal                     Arsenal  Home       1   
2021-08-14 12:30 Manchester UtdLeeds United      Leeds United  Home       1   
           15:00 BurnleyBrighton                     Brighton  Home      -1   
                 ChelseaCrystal Palace         Crystal Palace  Home       1   
                 EvertonSouthampton               Southampton  Home       1   
...                                                       ...   ...     ...   
2022-05-22 16:00 Crystal PalaceManchester Utd  Manchester Utd  Home       1   
                 Leicester CitySouthampton        Southampton  Home       1   
                 LiverpoolWolves                       Wolves  Home       1   
                 Manchester CityAston Villa       Aston Villa  Home       1   
                 Norwich CityTottenham              Tottenham  Home      -1   

                                               home_form_last_5  \
date       time  game                                             
2021-08-13 20:00 BrentfordArsenal                           NaN   
2021-08-14 12:30 Manchester UtdLeeds United                 NaN   
           15:00 BurnleyBrighton                            NaN   
                 ChelseaCrystal Palace                      NaN   
                 EvertonSouthampton                         NaN   
...                                                         ...   
2022-05-22 16:00 Crystal PalaceManchester Utd               0.2   
                 Leicester CitySouthampton                  0.0   
                 LiverpoolWolves                            0.8   
                 Manchester CityAston Villa                 0.8   
                 Norwich CityTottenham                     -0.8   

                                               home_avg_goals_last_5  \
date       time  game                                                  
2021-08-13 20:00 BrentfordArsenal                                NaN   
2021-08-14 12:30 Manchester UtdLeeds United                      NaN   
           15:00 BurnleyBrighton                                 NaN   
                 ChelseaCrystal Palace                           NaN   
                 EvertonSouthampton                              NaN   
...                                                              ...   
2022-05-22 16:00 Crystal PalaceManchester Utd                    1.2   
                 Leicester CitySouthampton                       2.2   
                 LiverpoolWolves                                 1.6   
                 Manchester CityAston Villa                      4.2   
                 Norwich CityTottenham                           0.2   

                                               home_avg_goals_conceded_last_5  \
date       time  game                                                           
2021-08-13 20:

In [8]:
# Get unique team names from the dataframe
team_names = df['home_team'].unique()

# Create a dictionary with team names as keys and 0 as values
team_points = {team: 0 for team in team_names}

In [9]:
mw1 = df_2021_2022[df_2021_2022['round'] == 'Matchweek 1']


In [10]:
for row in mw1.itertuples():
    if row.result == 1:
        team_points[row.home_team] += 3
    elif row.result == -1:
        team_points[row.away_team] += 3
    else:
        # Corrected logic for a draw
        team_points[row.home_team] += 1
        team_points[row.away_team] += 1

In [11]:
df_clean = df_2021_2022.dropna() # remove gameweek 1

In [12]:
not_features = ['round','home_team','away_team','venue','result']
features = [col for col in df_2021_2022.columns if col not in not_features]

In [13]:
X = df_clean[features]
y = df_clean['result']

In [14]:
cutoff_date = '2022-02-12'
df_train_period = df_clean[df_clean.index.get_level_values('date') <= cutoff_date] 
X_train = X[X.index.get_level_values('date') <= cutoff_date]
y_train = y[y.index.get_level_values('date') <= cutoff_date]
X_test = X[X.index.get_level_values('date') > cutoff_date]
y_test = y[y.index.get_level_values('date') > cutoff_date]

In [15]:
# Ensure X and y are aligned
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")
assert len(X_train) == len(y_train), "Training data misalignment!"
assert len(X_test) == len(y_test), "Test data misalignment!"

X_train shape: (219, 61)
y_train shape: (219,)
X_test shape: (151, 61)
y_test shape: (151,)


In [16]:
for row in df_train_period.itertuples():
    if row.result == 1: # Home win
        team_points[row.home_team] += 3
    elif row.result == -1: # Away win
        team_points[row.away_team] += 3
    else: # Draw
        team_points[row.home_team] += 1
        team_points[row.away_team] += 1

print("Initial Points Table (at start of test period):")

Initial Points Table (at start of test period):


In [17]:
sorted_initial_points = sorted(team_points.items(), key=lambda item: item[1], reverse=True)

In [18]:
print(pd.Series(dict(sorted_initial_points))) # League table after last day of train period

Manchester City    63
Liverpool          51
Chelsea            47
Manchester Utd     40
West Ham           40
Arsenal            39
Tottenham          36
Wolves             34
Brighton           33
Southampton        29
Aston Villa        27
Leicester City     26
Crystal Palace     26
Brentford          24
Leeds United       23
Everton            22
Newcastle Utd      18
Norwich City       17
Watford            15
Burnley            14
dtype: int64


In [19]:
rf_model = RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],       
    'max_depth': [None, 10, 20],          
    'min_samples_split': [2, 5, 10],      
    'min_samples_leaf': [1, 2, 4]         
}

tcsv = TimeSeriesSplit(n_splits=4)

In [20]:
grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    cv=tcsv,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1 
)

In [21]:
print("Starting hyperparameter tuning with Time Series Cross-Validation...")
grid_search.fit(X_train, y_train)
print("Tuning complete.")

Starting hyperparameter tuning with Time Series Cross-Validation...
Fitting 4 folds for each of 81 candidates, totalling 324 fits
Tuning complete.


In [22]:

print(f"Best Hyperparameters: {grid_search.best_params_}")

best_rf_model = grid_search.best_estimator_


final_predictions = best_rf_model.predict(X_test)
final_accuracy = accuracy_score(y_test, final_predictions)
final_f1 = f1_score(y_test, final_predictions, average='macro')

print(f"\nFinal Model Performance:")
print(f"Accuracy: {final_accuracy:.2%}")
print(f"Macro F1-Score: {final_f1:.2%}")

cm = confusion_matrix(y_test, final_predictions)
cm_df = pd.DataFrame(
    cm, 
    index=['Actual: Away Win', 'Actual: Draw', 'Actual: Home Win'],
    columns=['Pred: Away', 'Pred: Draw', 'Pred: Home']
)
cm_df


Best Hyperparameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}

Final Model Performance:
Accuracy: 52.32%
Macro F1-Score: 45.34%


,Pred: Away,Pred: Draw,Pred: Home
Actual: Away Win,23,10,22
Actual: Draw,7,6,12
Actual: Home Win,10,11,50


In [23]:
match_probabilities = best_rf_model.predict_proba(X_test)
prob_df = pd.DataFrame(match_probabilities, columns=best_rf_model.classes_, index=X_test.index)
prob_df.rename(columns={-1: 'away_win_prob', 0: 'draw_prob', 1: 'home_win_prob'}, inplace=True)
test_matches_with_probs = df_clean.loc[X_test.index, ['home_team', 'away_team']].join(prob_df)
test_matches_with_probs.head(15)

home_team       away_team  \
date       time  game                                                          
2022-02-13 14:00 BurnleyLiverpool                    Burnley       Liverpool   
                 Newcastle UtdAston Villa      Newcastle Utd     Aston Villa   
                 TottenhamWolves                   Tottenham          Wolves   
           16:30 Leicester CityWest Ham       Leicester City        West Ham   
2022-02-15 20:15 Manchester UtdBrighton       Manchester Utd        Brighton   
2022-02-19 12:30 West HamNewcastle Utd              West Ham   Newcastle Utd   
           15:00 ArsenalBrentford                    Arsenal       Brentford   
                 Aston VillaWatford              Aston Villa         Watford   
                 BrightonBurnley                    Brighton         Burnley   
                 Crystal PalaceChelsea        Crystal Palace         Chelsea   
                 LiverpoolNorwich City             Liverpool    Norwich City   
                 SouthamptonEverton              Southampton         Everton   
           17:30 Manchester CityTottenham    Manchester City       Tottenham   
2022-02-20 14:00 Leeds UnitedManchester Utd     Leeds United  Manchester Utd   
           16:30 WolvesLeicester City                 Wolves  Leicester City   

                                             away_win_prob  draw_prob  \
date       time  game                                                   
2022-02-13 14:00 BurnleyLiverpool                 0.602329   0.289655   
                 Newcastle UtdAston Villa         0.245419   0.391790   
                 TottenhamWolves                  0.345575   0.209706   
           16:30 Leicester CityWest Ham           0.352935   0.312524   
2022-02-15 20:15 Manchester UtdBrighton           0.271045   0.406756   
2022-02-19 12:30 West HamNewcastle Utd            0.259460   0.262558   
           15:00 ArsenalBrentford                 0.106086   0.134745   
                 Aston VillaWatford               0.228838   0.266007   
                 BrightonBurnley                  0.248986   0.274962   
                 Crystal PalaceChelsea            0.272683   0.320849   
                 LiverpoolNorwich City            0.165567   0.204516   
                 SouthamptonEverton               0.425215   0.250542   
           17:30 Manchester CityTottenham         0.268044   0.191582   
2022-02-20 14:00 Leeds UnitedManchester Utd       0.604368   0.214820   
           16:30 WolvesLeicester City             0.149908   0.204955   

                                             home_win_prob  
date       time  game                                       
2022-02-13 14:00 BurnleyLiverpool                 0.108015  
                 Newcastle UtdAston Villa         0.362792  
                 TottenhamWolves                  0.444719  
           16:30 Leicester CityWest Ham           0.334541  
2022-02-15 20:15 Manchester UtdBrighton           0.322198  
2022-02-19 12:30 West HamNewcastle Utd            0.477982  
           15:00 ArsenalBrentford                 0.759170  
                 Aston VillaWatford               0.505155  
                 BrightonBurnley                  0.476052  
                 Crystal PalaceChelsea            0.406468  
                 LiverpoolNorwich City            0.629917  
                 SouthamptonEverton               0.324243  
           17:30 Manchester CityTottenham         0.540374  
2022-02-20 14:00 Leeds UnitedManchester Utd       0.180813  
           16:30 WolvesLeicester City             0.645137

In [66]:
start_points = dict(sorted_initial_points)

def single_season_sim(league_table, matches_to_play):
    sim_points = league_table.copy()
    outcomes = [-1, 0, 1]  # -1: Away Win, 0: Draw, 1: Home Win
    
    # Iterate through each match in the test set schedule
    for index, match in matches_to_play.iterrows():
        home_team = match['home_team']
        away_team = match['away_team']
        
        # Get the probabilities for the three outcomes for this specific match
        probs = [match['away_win_prob'], match['draw_prob'], match['home_win_prob']]
        
        # "Roll the dice": Choose an outcome based on the model's probabilities
        simulated_result = np.random.choice(outcomes, p=probs)
        
        # Update the points table based on the simulated result
        if simulated_result == 1:  # Home win
            sim_points[home_team] += 3
        elif simulated_result == -1:  # Away win
            sim_points[away_team] += 3
        else:  # Draw
            sim_points[home_team] += 1
            sim_points[away_team] += 1
            
    return sim_points

print("Running one sample simulation...")
one_final_table = single_season_sim(start_points, test_matches_with_probs)

sorted_final_table = sorted(one_final_table.items(), key=lambda item: item[1], reverse=True)
print("\nResult of one simulated season:")
print(pd.Series(dict(sorted_final_table)))



Running one sample simulation...

Result of one simulated season:
Manchester City    88
Liverpool          81
Chelsea            70
Arsenal            69
Manchester Utd     63
Wolves             60
Tottenham          58
Leicester City     54
Brighton           53
West Ham           50
Southampton        48
Brentford          47
Crystal Palace     45
Newcastle Utd      44
Everton            42
Burnley            37
Aston Villa        36
Leeds United       32
Norwich City       31
Watford            29
dtype: int64


In [ ]:
simulations = {}

for x in range(1000):
    sim = single_season_sim(start_points, test_matches_with_probs)
    simulations[x] = sim # running 1000 Monte-Carlo experiments on the season

In [73]:
simulations

{0: {'Manchester City': 92,
  'Liverpool': 90,
  'Chelsea': 70,
  'Manchester Utd': 54,
  'West Ham': 57,
  'Arsenal': 57,
  'Tottenham': 61,
  'Wolves': 56,
  'Brighton': 52,
  'Southampton': 40,
  'Aston Villa': 53,
  'Leicester City': 55,
  'Crystal Palace': 56,
  'Brentford': 38,
  'Leeds United': 40,
  'Everton': 40,
  'Newcastle Utd': 28,
  'Norwich City': 26,
  'Watford': 34,
  'Burnley': 38},
 1: {'Manchester City': 89,
  'Liverpool': 80,
  'Chelsea': 69,
  'Manchester Utd': 67,
  'West Ham': 50,
  'Arsenal': 70,
  'Tottenham': 69,
  'Wolves': 56,
  'Brighton': 47,
  'Southampton': 45,
  'Aston Villa': 37,
  'Leicester City': 48,
  'Crystal Palace': 49,
  'Brentford': 33,
  'Leeds United': 35,
  'Everton': 45,
  'Newcastle Utd': 34,
  'Norwich City': 34,
  'Watford': 33,
  'Burnley': 50},
 2: {'Manchester City': 88,
  'Liverpool': 84,
  'Chelsea': 73,
  'Manchester Utd': 70,
  'West Ham': 56,
  'Arsenal': 60,
  'Tottenham': 55,
  'Wolves': 59,
  'Brighton': 51,
  'Southampton':

In [ ]:
simulations_df = pd.DataFrame()
for sim_num, table in simulations.items():
    simulations_df[f"sim_{sim_num}"] = pd.Series(table) # Made each simulation a column in a dataframe


In [ ]:
simulations_df

sim_0  sim_1  sim_2  sim_3  sim_4  sim_5  sim_6  sim_7  \
Manchester City     92     89     88     92     95     82     88     92   
Liverpool           90     80     84     81     79     89     76     78   
Chelsea             70     69     73     70     75     70     78     68   
Manchester Utd      54     67     70     68     54     61     60     57   
West Ham            57     50     56     54     61     57     56     54   
Arsenal             57     70     60     63     62     73     64     68   
Tottenham           61     69     55     61     61     63     62     58   
Wolves              56     56     59     61     53     49     58     53   
Brighton            52     47     51     52     55     53     49     51   
Southampton         40     45     50     45     50     50     48     40   
Aston Villa         53     37     41     43     54     41     43     51   
Leicester City      55     48     50     51     42     52     56     45   
Crystal Palace      56     49     43     42     45     48     47     53   
Brentford           38     33     44     48     39     45     44     46   
Leeds United        40     35     33     38     40     39     37     42   
Everton             40     45     36     39     33     40     45     31   
Newcastle Utd       28     34     52     48     33     37     28     44   
Norwich City        26     34     30     24     29     25     26     28   
Watford             34     33     27     22     34     26     25     40   
Burnley             38     50     33     32     34     34     45     34   

                 sim_8  sim_9  sim_10  sim_11  sim_12  sim_13  sim_14  sim_15  \
Manchester City     94     95      82      92      97      85      88      91   
Liverpool           80     81      84      74      88      84      76      83   
Chelsea             63     66      69      72      66      72      73      69   
Manchester Utd      60     55      65      55      58      59      63      64   
West Ham            60     52      64      56      60      56      55      64   
Arsenal             62     74      64      64      72      59      66      69   
Tottenham           62     61      65      59      54      67      60      66   
Wolves              56     55      43      53      60      50      50      57   
Brighton            49     49      46      53      53      50      50      57   
Southampton         44     52      43      43      45      48      45      42   
Aston Villa         49     41      45      49      50      50      43      37   
Leicester City      52     57      55      44      47      54      52      46   
Crystal Palace      50     42      44      55      36      40      41      49   
Brentford           39     40      40      39      41      47      40      37   
Leeds United        41     35      35      37      36      41      41      37   
Everton             31     35      48      34      39      38      45      38   
Newcastle Utd       34     37      41      44      31      40      40      37   
Norwich City        34     32      34      36      36      24      35      28   
Watford             28     39      31      32      30      29      20      29   
Burnley             44     32      32      36      35      41      39      37   

                 sim_16  sim_17  sim_18  sim_19  sim_20  sim_21  sim_22  \
Manchester City      87      84      98      96      87      91      91   
Liverpool            94      79      76      88      87      83      77   
Chelsea              73      83      74      69      75      69      74   
Manchester Utd       53      57      57      57      55      57      60   
West Ham             55      56      52      47      63      60      56   
Arsenal              57      66      65      60      65      66      64   
Tottenham            62      57      51      65      55      73      56   
Wolves               52      52      57      64      52      52      44   
Brighton             52      54      56      49      54      49 

In [84]:
simulations_df.to_csv("../data/04_simulated_seasons/2021_2022_sim.csv")